In [1]:
import tkinter as tk
from tkinter import font
import serial
from serial.tools import list_ports
from PIL import Image, ImageTk
import numpy as np
import matplotlib
matplotlib.use("TkAgg")
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2Tk
from matplotlib.figure import Figure
import seaborn as sns
import struct

In [4]:
class Application(tk.Frame):
    def __init__(self, master=None):
        super().__init__(master)
        self.master = master
        self.pack()
        self.create_widgets()
        self.master.title("STAG")
        self.master.geometry("{0}x{1}+0+0".format(master.winfo_screenwidth(), master.winfo_screenheight()))
        self.master.protocol("WM_DELETE_WINDOW", self.on_exit)        
        self.serial_port = serial.Serial(list_ports.comports()[0].device, timeout=0.5, baudrate=115200)
        self.poll_serial()
        
    def on_exit(self):
        self.serial_port.close()
        self.master.quit()
        self.master.destroy()
    
    def create_widgets(self):
        self.max = 0
        
        self.fig = Figure(figsize=(6,6), dpi=100)
        self.ax = self.fig.add_subplot(111)
        data = np.ones((32,32))
        self.mask = np.array([np.ones(32), np.ones(32), np.ones(32),
                         np.concatenate((np.zeros(14), np.ones(18))),
                         np.concatenate((np.zeros(14), np.ones(18))),
                         np.concatenate((np.zeros(14), np.ones(18))),
                         np.ones(32), np.ones(32), np.ones(32),
                         np.concatenate((np.zeros(14), np.ones(18))),
                         np.ones(32), np.ones(32), np.ones(32),
                         np.concatenate((np.zeros(14), np.ones(18))),
                         np.concatenate((np.zeros(14), np.ones(18))),
                         np.ones(32), np.ones(32), np.ones(32),
                         np.concatenate((np.zeros(25), np.ones(4), np.zeros(3))),
                         np.concatenate((np.zeros(25), np.ones(4), np.zeros(3))),
                         np.concatenate((np.zeros(25), np.ones(4), np.zeros(3))),
                         np.concatenate((np.zeros(25), np.ones(4), np.zeros(3))),
                         np.concatenate((np.zeros(25), np.ones(4), np.zeros(3))),
                         np.concatenate((np.zeros(25), np.ones(4), np.zeros(3))),
                         np.concatenate((np.zeros(25), np.ones(4), np.zeros(3))),
                         np.concatenate((np.zeros(25), np.ones(4), np.zeros(3))),
                         np.concatenate((np.zeros(25), np.ones(4), np.zeros(3))),
                         np.concatenate((np.zeros(25), np.ones(4), np.zeros(3))),
                         np.concatenate((np.zeros(25), np.ones(4), np.zeros(3))),
                         np.concatenate((np.zeros(25), np.ones(4), np.zeros(3))),
                         np.concatenate((np.zeros(25), np.ones(4), np.zeros(3))),
                         np.concatenate((np.zeros(25), np.ones(4), np.zeros(3)))]).astype(np.bool)
        data[np.logical_not(self.mask)] = 0
        sns.heatmap(data, square=True, ax=self.ax, cbar=False)
        self.ax.title.set_text('Sensor data')
        self.canvas = FigureCanvasTkAgg(self.fig, self)
        self.canvas.draw()
        self.canvas.get_tk_widget().grid(row=0, column=0)

        #self.toolbar = NavigationToolbar2Tk(self.canvas, self.leftframe)
        #self.toolbar.update()
        #self.canvas._tkcanvas.grid(row=1, column=0)#(side=tk.TOP, fill=tk.BOTH, expand=True)
        
        self.rightframe = tk.Frame(self)
        self.rightframe.grid(row=0, column=1)
        
        self.var = tk.IntVar(self)
        
        self.pause = tk.Radiobutton(self.rightframe, variable=self.var, indicatoron=0, width=6, text="Pause", value=1, command=self.send_pause)
        self.pause.grid(row=0, column=0)
        self.pause.select()
        self.paused = 1
        
        self.run = tk.Radiobutton(self.rightframe, variable=self.var, indicatoron=0, width=6, text=" Run ", value=2, command=self.send_run)
        self.run.grid(row=1, column=0)
        
        self.clear = tk.Button(self.rightframe, text="Clear Max", command=self.clear_max)
        self.clear.grid(row=2, column=0)
        
        self.text = tk.Text(self.rightframe, width=20, height=1, wrap=tk.WORD)
        self.text.insert(tk.END, "")
        self.text.grid(row=3, column=0)
            

    def send_pause(self):
        if(self.paused == 0):
            self.serial_port.write("p".encode())
            self.serial_port.flush()
            self.paused = 1
            

    def send_run(self):
        if(self.paused == 1):
            self.serial_port.write("r".encode())
            self.serial_port.flush()
            self.paused = 0
            
            
    def clear_max(self):
        self.max = 0
        self.text.delete("1.0", tk.END)
        self.text.insert(tk.END, self.max)

            
    def poll_serial(self):
        data = self.serial_port.readline()
        data = data.decode()
        #data = np.random.random((32, 32))*100
        if (len(data) > 0):#len(data) >= 1024):
            # Data is a sensor frame
            # Draw on canvas
            data = data.split('\t')
            data.pop(-1)
            try:
                data = np.array(data).astype(np.float)
                current_max = np.amax(data)
                if (self.max < current_max < 4096):
                    self.max = current_max
                data = np.reshape(data, (32,32))                
                sns.heatmap(data, square=True, ax=self.ax, cbar=False)#, vmax=2300, vmin=1820)
                self.canvas.draw()
            except Exception as e: 
                print(e)
                #print(data)
            self.text.delete("1.0", tk.END)
            self.text.insert(tk.END, self.max)
        #elif (0 < len(data) > 10):
            # Data is prediction
            #print(len(data))
            #self.text.delete("1.0", tk.END)
            #self.text.insert(tk.END, data)
        
        self.master.after(1, self.poll_serial)

In [5]:
root = tk.Tk()

default_font = font.nametofont("TkDefaultFont")
default_font.configure(size=root.winfo_screenwidth()//90)
text_font = font.nametofont("TkTextFont")
text_font.configure(size=root.winfo_screenwidth()//90)
fixed_font = font.nametofont("TkFixedFont")
fixed_font.configure(size=root.winfo_screenwidth()//90)
#print(root.winfo_screenwidth()//100)

app = Application(master=root)
app.mainloop()

cannot reshape array of size 230 into shape (32,32)
cannot reshape array of size 964 into shape (32,32)
cannot reshape array of size 912 into shape (32,32)
cannot reshape array of size 893 into shape (32,32)
cannot reshape array of size 986 into shape (32,32)
cannot reshape array of size 841 into shape (32,32)
cannot reshape array of size 827 into shape (32,32)
cannot reshape array of size 779 into shape (32,32)
cannot reshape array of size 2570 into shape (32,32)
cannot reshape array of size 777 into shape (32,32)
cannot reshape array of size 745 into shape (32,32)
cannot reshape array of size 1774 into shape (32,32)
cannot reshape array of size 969 into shape (32,32)
cannot reshape array of size 656 into shape (32,32)


In [ ]:
import pyqtgraph as pg